In [2]:
# Import required libraries
import pandas as pd
import numpy as np

# Read in the data
df = pd.read_csv('nigeria_agricultural_exports.csv')
df.head()

,Product Name,Company,Export Country,Date,Units Sold,unit_price,Profit per unit,Export Value,Destination Port,Transportation Mode
0,Rubber,Farmgate Nigeria Limited,Austria,11/16/2023,721,31443.05,5863.92,22670439.05,Lagos,Sea
1,Palm Oil,Prime Agro Exports Nigeria Limited,Germany,1/8/2021,881,23151.97,5868.73,20396885.57,Lagos,Sea
2,Cassava,Farmgate Nigeria Limited,Germany,10/14/2020,702,18536.45,5105.01,13012587.90,Calabar,Sea
3,Rubber,Nigerian Export Promotion Council (NEPC),Belgium,12/31/2022,191,21981.31,7781.54,4198430.21,Warri,Sea
4,Sesame,Nigeria Agro Export Company,France,12/19/2022,373,13415.94,9590.95,5004145.62,Lagos,Sea


In [3]:
# Examine data for nulls and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Product Name         1000 non-null   object 
 1   Company              1000 non-null   object 
 2   Export Country       1000 non-null   object 
 3   Date                 1000 non-null   object 
 4   Units Sold           1000 non-null   int64  
 5   unit_price           1000 non-null   float64
 6   Profit per unit      1000 non-null   float64
 7   Export Value         1000 non-null   float64
 8   Destination Port     1000 non-null   object 
 9   Transportation Mode  1000 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 78.2+ KB


# Data Cleaning and Manipulation

In [4]:
# Replace / with - in the date column
df['Date'] = df['Date'].str.replace('/', '-')

# Convert Date column to date data type, the dates are in the mm-dd-yyyy format
df['Date'] = pd.to_datetime(df['Date'], format = '%m-%d-%Y')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Product Name         1000 non-null   object        
 1   Company              1000 non-null   object        
 2   Export Country       1000 non-null   object        
 3   Date                 1000 non-null   datetime64[ns]
 4   Units Sold           1000 non-null   int64         
 5   unit_price           1000 non-null   float64       
 6   Profit per unit      1000 non-null   float64       
 7   Export Value         1000 non-null   float64       
 8   Destination Port     1000 non-null   object        
 9   Transportation Mode  1000 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 78.2+ KB


In [6]:
# Create profit column (Units Sold * profit per unit)
df['Profit'] = df['Units Sold'] * df['Profit per unit']

# Create cogs column (export value - profit)
df['Cost of goods sold'] = df['Export Value'] - df['Profit']

# Create profit margin column (profit / export value), rounded to 2 decimal places
df['Profit Margin'] = (df['Profit'] / df['Export Value']).round(2)

df.head()

,Product Name,Company,Export Country,Date,Units Sold,unit_price,Profit per unit,Export Value,Destination Port,Transportation Mode,Profit,Cost of goods sold,Profit Margin
0,Rubber,Farmgate Nigeria Limited,Austria,2023-11-16,721,31443.05,5863.92,22670439.05,Lagos,Sea,4227886.32,18442552.73,0.19
1,Palm Oil,Prime Agro Exports Nigeria Limited,Germany,2021-01-08,881,23151.97,5868.73,20396885.57,Lagos,Sea,5170351.13,15226534.44,0.25
2,Cassava,Farmgate Nigeria Limited,Germany,2020-10-14,702,18536.45,5105.01,13012587.90,Calabar,Sea,3583717.02,9428870.88,0.28
3,Rubber,Nigerian Export Promotion Council (NEPC),Belgium,2022-12-31,191,21981.31,7781.54,4198430.21,Warri,Sea,1486274.14,2712156.07,0.35
4,Sesame,Nigeria Agro Export Company,France,2022-12-19,373,13415.94,9590.95,5004145.62,Lagos,Sea,3577424.35,1426721.27,0.71


In [7]:
df.Company.unique()

array(['Farmgate Nigeria Limited', 'Prime Agro Exports Nigeria Limited',
       'Nigerian Export Promotion Council (NEPC)',
       'Nigeria Agro Export Company',
       'Greenfield Agro Exporters Nigeria Limited',
       'Solid Agro Nigeria Limited', 'Agro Export Nigeria Ltd',
       'Golden Farms Nigeria Limited', 'Agriplus Nigeria Limited',
       'Agro Allied Exporters Nigeria Limited'], dtype=object)

In [8]:
# Define a function to abbreviate the company names
def abbreviate(names):
    """Abbreviate each name in a pandas Series of names.

    This function takes a pandas Series of full names and returns a Series of their abbreviations.
    Each abbreviation is formed by taking the first letter of each word in the name and concatenating them."""
    
    return names.apply(lambda x: ''.join([word[0] for word in x.split()]))

# Test the function
sample = pd.Series(['Chelsea Football Club', 'Wolverhampton Wonderers Football Club', 'Newcastle United'])
print(abbreviate(sample))

0     CFC
1    WWFC
2      NU
dtype: object


In [9]:
# Create a new column for the abbreviation of the company names
df['company_short'] = abbreviate(df['Company'])

# Remove the open bracket after NEPC in the company_short column
df.loc[df['company_short'] == 'NEPC(', 'company_short'] = 'NEPC'

df.head()

,Product Name,Company,Export Country,Date,Units Sold,unit_price,Profit per unit,Export Value,Destination Port,Transportation Mode,Profit,Cost of goods sold,Profit Margin,company_short
0,Rubber,Farmgate Nigeria Limited,Austria,2023-11-16,721,31443.05,5863.92,22670439.05,Lagos,Sea,4227886.32,18442552.73,0.19,FNL
1,Palm Oil,Prime Agro Exports Nigeria Limited,Germany,2021-01-08,881,23151.97,5868.73,20396885.57,Lagos,Sea,5170351.13,15226534.44,0.25,PAENL
2,Cassava,Farmgate Nigeria Limited,Germany,2020-10-14,702,18536.45,5105.01,13012587.90,Calabar,Sea,3583717.02,9428870.88,0.28,FNL
3,Rubber,Nigerian Export Promotion Council (NEPC),Belgium,2022-12-31,191,21981.31,7781.54,4198430.21,Warri,Sea,1486274.14,2712156.07,0.35,NEPC
4,Sesame,Nigeria Agro Export Company,France,2022-12-19,373,13415.94,9590.95,5004145.62,Lagos,Sea,3577424.35,1426721.27,0.71,NAEC


In [10]:
df.to_csv('cleaned_agric_exports.csv', index=True)